In [31]:
import numpy as np

# V Belt Design Script

Follow the instructions. Thank Felix.

Felix takes no responsability for marks lost due to errors in this notebook, Fuck you.

Provided constants go here: 

In [32]:
# FILL IN ______________________________

H_in = 50   # horsepower

# shaft speeds. n1 is the faster shaft.
n1 = 1160   # RPM
n2 = 600    # RPM

In [33]:
# use this function to linearly interpolate from graphs
# known points are (x1, y1) and (x2, y2)
# function returns the interpolated value associated with x3
def linear_interpolate(x1, y1, x2, y2, x3):
    # Ensure x1 and x2 are not the same to avoid division by zero
    if x1 == x2:
        raise ValueError("x1 and x2 cannot be the same value.")
    
    y3 = y1 + (y2 - y1) * ((x3 - x1) / (x2 - x1))
    return y3

### Select a service factor from this table: 

![V Belt Image](./images/V_Belt_SF.png)

In [34]:
# FILL IN ______________________________
SF = 1.5

P_des = H_in * SF

print(f'Design horsepower: {P_des}')

Design horsepower: 75.0


### Select Belt type from this table
Use the design horsepower printed above. If at the boundary between two different types of belts, be conservative and choose the larger
one.

![V Belt Image](./images/V_Belt_Type.png)

In [35]:
VR_guess = n1 / n2 

print(f'Velocity Ratio = {VR_guess}')

Velocity Ratio = 1.9333333333333333


### Select the driving sheave size to produce a belt speed of 4000 ft/min 
(this is a standard speed we use since belt speed should not surpass 5000 ft/min, with a hard max at 6500 ft/min)

In [36]:
# adjust this value following the above guidlines if necessary. 4000 is default
v_b_guess = 4000

# driving sheave diameter
D1_guess = v_b_guess * 2 * 12 / n1 / (2 * np.pi)

print(f'Estimated small sheave diameter = {D1_guess} inches')

Estimated small sheave diameter = 13.171443566225822 inches


### Select the appropriate small sheave diameter from the following tables. Select the closest value to the estimate above


#### 3V belts
![V Belt Image](./images/3V_DriveSheave.png)

#### 5V belts
![V Belt Image](./images/5V_DriveSheave.png)

#### 8V belts
![V Belt Image](./images/8V_DriveSheave.png)

In [37]:
# FILL IN ______________________________
D1 = 13.1
# FILL IN ______________________________
P_rated = 28

In [38]:
D2_guess = D1 * VR_guess

print(f'Estimated D2 = {D2_guess} inches')
print(f'select the appropriate standard D2 from the table above')

Estimated D2 = 25.326666666666664 inches
select the appropriate standard D2 from the table above


In [39]:
# FILL IN ______________________________
D2 = 27.7


VR = D2 / D1
print(f'Actual Speed ratio VR: {VR}')

v_b = D1 * n1 * 2 * np.pi / 2 / 12
print(f'belt speed: {v_b} ft/min')
print('check that this value is around 4000 and must be less then 6500 ft/min')

Actual Speed ratio VR: 2.114503816793893
belt speed: 3978.303496995875 ft/min
check that this value is around 4000 and must be less then 6500 ft/min


### If the actual speed ratio is higher than 1, use the following table to find the power added
The lines are the speed of the small shaft
![V Belt Image](./images/P_added.png)

In [40]:
# example using linear interpolate to get value from table above
print(linear_interpolate(1750, 1.9, 1160, 1.26, n1))

1.26


In [41]:


# FILL IN ______________________________
P_added = 1.26



P_rated_total = P_added + P_rated

print(f'Total rated power: {P_rated_total}')

Total rated power: 29.26


### We now estimate a trial center distance based on this equation
The formula below will output a trial center distance in the exact center of this range

![V Belt Image](./images/Trial_CD.png)

In [42]:
CD_upperbound = 3 * (D2 + D1)
CD_lowerbound = D2

print(f'Cd must be between: {CD_lowerbound} and {CD_upperbound}')

# adjust CD_trial manually if you want to
CD_trial = (CD_upperbound + CD_lowerbound) / 2

# you can just hard code the value of CD_trial if needed like below
# CD_trial = 43

print(f'Trial center distance: {CD_trial}')

Cd must be between: 27.7 and 122.39999999999999
Trial center distance: 75.05


### Now we calculate LP from this formula

![V Belt Image](./images/Lp_Formula.png)

In [43]:
Lp_guess = 2*CD_trial + 1.57*(D2 + D1) + (D2 - D1)**2 / (4 * CD_trial)

print(f'Length from trial distance: {Lp_guess}')

Length from trial distance: 214.86605996002666


### Choose the belt length from this table

![V Belt Image](./images/belt_length.png)

### Now we calculate the actual center distance from these formulas
Fill in the selected belt length below

![V Belt Image](./images/center_distance.png)

In [44]:
# FILL IN ______________________________
Lp = 212


B = 4 * Lp - 6.28*(D1 + D2)


CD = (B + np.sqrt(B**2 - 32 * (D2 - D1)**2)) / 16

print(f'center distance: {CD} inches')



center distance: 73.61002484708638 inches


### Wrap angle calculations 

![V Belt Image](./images/small_wrap.png)

![V Belt Image](./images/big_wrap.png)

In [45]:
theta_1 = 180 - 2 * np.degrees(np.arcsin((D2 - D1) / (2 * CD)))

theta_2 = 180 + 2 * np.degrees(np.arcsin((D2 - D1) / (2 * CD)))


print(f'Small sheave wrap angle: {theta_1} degrees')

print(f'Big sheave wrap angle: {theta_2} degrees')

Small sheave wrap angle: 168.61709839378625 degrees
Big sheave wrap angle: 191.38290160621375 degrees


### Find these randomass correction factors
Use the small sheave for the wrap correction factor

![V Belt Image](./images/C_theta.png)

![V Belt Image](./images/C_Lp.png)

In [51]:
linear_interpolate(169, 0.95, 180, 1, 168.62)

0.9482727272727273

In [46]:
# FILL IN ______________________________
C_theta = 0.948
C_Lp = 1.1

In [60]:
corrected_power_rating = C_theta * C_Lp * P_rated_total
print(f'corrected power rating: {corrected_power_rating} HP')

num_belts = P_des / corrected_power_rating

safety_factor = corrected_power_rating / P_des * np.ceil(num_belts)

print(f'we need {num_belts} belts')
print(f'round up! We need {np.ceil(num_belts)}')
print(f'safety factor: {safety_factor}')

corrected power rating: 28.164213000000004 HP
we need 2.6629538698631485 belts
round up! We need 3.0
safety factor: 1.1265685200000002


In [52]:
print(f'State you belt type: 3V, 5V, 8V')
print(f'D1: {D1} inches')
print(f'D2: {D2} inches')
print(f'VR: {VR}')
print(f'CD: {CD}')
print(f'{np.ceil(num_belts)} belts needed')

State you belt type: 3V, 5V, 8V
D1: 13.1 inches
D2: 27.7 inches
VR: 2.114503816793893
CD: 73.61002484708638
3.0 belts needed
